In [2]:
from selenium import webdriver
import time
import requests
import pandas as pd
import openpyxl
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re
import numpy as np
from tqdm import tqdm_notebook

from datetime import date
from datetime import timedelta


from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import *

In [14]:
start_time = time.time()

#rakuten_id_list = []
#title_list = []
#year_list = []
#time_list = []
#overview_list = []
#director_list = []
#starring_list = []
#url_list = []
#genre_list = []
#studio_list = []


rakuten_df = pd.DataFrame(columns = ['rakuten_id', 'title', 'release', 'genre', 'director','actor', 'country','runtime','production','overview', 'url'])
    
titles = []
urls = []
rakuten_ids = []
overview = []
country = []
genres = []
year = []


path_ = '../../chromedriver/chromedriver'

url = 'https://www.viki.com/explore?locale=en'

driver = webdriver.Chrome(path_)

driver.get(url)

time.sleep(7)

# 영화 리스트만 선택

driver.find_element_by_xpath('//*[@id="select2-chosen-2"]').click()
time.sleep(1)
driver.find_element_by_xpath('//*[@id="select2-result-label-9"]').click()
time.sleep(5)

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
time.sleep(1)
driver.find_element_by_xpath('//*[@id="pjaxify-container"]/div[2]/div[1]/div/div[2]/div[2]/div[2]/a[6]').click()
time.sleep(5)

# 최대 페이지 수 가져오기

max_page_url = driver.current_url
max_page_loc_1 = max_page_url.find('page=')
max_page_loc_2 = max_page_url.find('&type')
max_page = int(max_page_url[max_page_loc_1 + 5:max_page_loc_2])
print('max page number : ' + str(max_page))



# 처음부터 최대 페이지까지 반복

for i in range(1, max_page + 1):
    url = 'https://www.viki.com/explore?page={}&sort=views_recent&type=film'.format(i)
    
    driver.get(url)
    time.sleep(2)

    print('page number ' + str(i) + ' crawling started...')
    
    soup = BeautifulSoup(driver.page_source, 'lxml')

    movies = soup.find_all('a', class_ = 'thumb-title strong')

    time.sleep(5)

    for movie in movies:
        
        # 주소
        
        href = (movie['href'])
        url = 'https://www.viki.com/' + href
        
        driver.get(url)
        time.sleep(3)
        
        # 무료가 아닌 것들, 혹은 그냥 볼 수 없는 영화들을 걸러냄
        
        soup2 = BeautifulSoup(driver.page_source, 'lxml')
        if soup2.find('h2', class_ = 'section-heading left') != None and soup2.find('div', class_ = 'center-align alert-container') == None:
            rakuten_id = href.replace('/movies/', '').split('-')[0]
            
            # 제목
            
            title = soup2.find('h1', class_ = 'display-2 strong').text.strip()
            
            titles.append(title)
            urls.append(url)
            rakuten_ids.append(rakuten_id)
            
            # 국가, 요약
            
            time.sleep(3)
            overview.append(soup2.find('p', class_= 'js-text-truncate').text)
            time.sleep(3)
            country.append(soup2.find('a', class_='badge badge-medium badge-gutter badge-regular').text)
            genre = soup2.find_all('a', itemprop='genre')
        
            dl_list = soup2.find('dl', {'class':'dl-horizontal'})
            dt_list = dl_list.find_all('dt')
            dd_list = dl_list.find_all('dd')
            
            # 개봉 년도
            
            dt_counter = 0
            
            for i in range(len(dt_list)):
                if dt_list[i].text == 'Broadcast Period':
                    year_reverse = dd_list[i].text
                    
                    year_month = year_reverse[:2]
                    year_day = year_reverse[3:5]
                    year_year = year_reverse[6:]
                    
                    year_correct = year_year + '-' + year_month + '-' + year_day
                    year.append(year_correct)
                    
                    dt_counter += 1
            if dt_counter == 0:
                year.append('None')
        
            # 장르
        
            temp = []
            for a in genre:
                temp.append(a.text)
                
            if len(temp) > 0:
                genre_temp = temp[0]
                if len(temp) > 1:
                    for i in temp[1:]:
                        genre_temp = genre_temp + ',' + i
            genres.append(genre_temp.strip())
        
    
    
print()
print('title len : ' + str(len(titles)))
print('id len : ' + str(len(rakuten_ids)))
print('year len : ' + str(len(year)))
print('url len : ' + str(len(urls)))
print('country len : ' + str(len(country)))
print('overview len : ' + str(len(overview)))
print('genre len : ' + str(len(genres)))
print()


rakuten_df['rakuten_id'] = rakuten_ids
rakuten_df['title'] = titles
rakuten_df['release'] = year
rakuten_df['url'] = urls
rakuten_df['url'].replace('\'\'', '')
    
rakuten_df['country'] = country
rakuten_df['overview'] = overview
rakuten_df['genre'] = genres



rakuten_df.to_excel('Rakuten_Crawling.xlsx') 

print('total time : ', time.time() - start_time)
    
    
driver.close()




max page number : 8
page number 1 crawling started...
page number 2 crawling started...
page number 3 crawling started...
page number 4 crawling started...
page number 5 crawling started...
page number 6 crawling started...
page number 7 crawling started...
page number 8 crawling started...

title len : 125
id len : 125
year len : 125
url len : 125
country len : 125
overview len : 125
genre len : 125

total time :  1826.2457430362701
